In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Q. mnist 데이터 셋에 대해서 Pytorch를 적용하여 모델 구성 변경, 조기 학습 중단을 적용하여 학습하고 Best model을 저장한 후 다시 불러와서 테스트 데이터로 평가한 결과를 출력하세요.

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_size = int(0.8 + len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=64, shuffle=False)
testloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

class MyModel(nn.Module):
  def __init__(self):
      super(MyModel, self).__init__()
      self.conv1 = nn.Conv2d(1, 20, 5)
      self.pool = nn.MaxPool2d(2, 2)
      self.flatten = nn.Flatten()
      self.fc1 = nn.Linear(2880, 50)
      self.fc2 = nn.Linear(50, 10)

  def forward(self, x):
      x = self.pool(F.relu(self.conv1(x)))
      x = self.flatten(x)
      x = F.relu(self.fc1(x))
      x = self.fc2(x)
      return x

model = MyModel()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

best_val_loss = float('inf')
patience, trials = 5.0
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

val_loss = 0.0
model.eval()
with torch.no_grad():
    for inputs, labels in valloader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        val_loss += loss.item()

val_loss /= len(valloader)
print(f'Epoch {epoch+1}, Train Loss: {running_loss / len(trainloader)}, Val Loss: {val_loss}')

if val_loss < best_val_loss:
    print(f'Validation Loss Decreased({best_val_loss:.6f}--->{val_loss:.6f}) \t Saving The Model')
    best_val_loss = val_loss
    trials = 0
    save_path = ""
    torch.save(model.state_dict(), save_path)
else:
    trials += 1
    if trials >= patience:
        print("Early stopping triggered")
        break

model.load_state_dict(torch.load(save_path))

correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the 10000 test images: {accuracy}%')

In [2]:
import torch
from torch.utils.data import DataLoader, random_split
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# from torchsummary import summary # 지금 여기서는 필요 없어서 안해도 됨

# MNIST 데이터셋을 위한 전처리 과정 정의
# 모델의 학습 효율성을 향상시키고, 일반적으로 신경망에서 더 나은 성능을 얻기 위해 널리 사용
transform = transforms.Compose([
    transforms.ToTensor(),  # 높이, 너비, 채널(예: RGB)의 3차원 배열을 0과 1 사이의 값으로 스케일링된 텐서로 변경
    transforms.Normalize((0.5,), (0.5,))  # 이미지를 평균 0.5, 표준편차 0.5로 정규화하여 [-1, 1] 범위로 조정
]) # 평균 0 표준편차 1 로 하면 아마 [-3, 3] 범위

# MNIST 데이터셋 로드
# transform=transform: 이미지에 적용할 전처리 과정을 지정
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
# data디렉터리에 저장(ls로확인가능), 학습이냐아니냐, 다운로드 할거고, 텐서로 바꾸고 -1 부터 1 범위로 조정 하겠다.

# 훈련 데이터셋을 훈련 및 검증 세트로 분할
train_size = int(0.8 * len(train_dataset))  # 훈련 세트 크기를 전체의 80%로 설정 # 개수니까 정수로 해야해서int.(부동소수점 버림)
val_size = len(train_dataset) - train_size  # 검증 세트 크기 계산
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])  # 분할 실행

# DataLoader는 데이터셋에서 미니 배치를 자동으로 생성하고, 이를 모델 학습이나 평가에 이용할 수 있게 해주는 유틸리티
# 여기서 데이터포인트 단위로 되어있어서 shuffle=True 를 trainloader에서 하는데 .. 검증이나 테스트는 평가하기 위한거니까 할 필요 없고.
# batch_size =64 -> 여기가 안에 들어갈 데이터포인트들이 셔플되는것임
trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True) # 데이터를 로드하기 전에 데이터셋을 무작위로 섞기
valloader = DataLoader(val_dataset, batch_size=64, shuffle=False) # 데이터의 순서가 성능 평가에 영향을 미치지 않는다.
testloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 모델 아키텍처 정의
# MyModel 클래스는 1채널 그레이스케일 이미지를 입력으로 받아, 10개의 출력 클래스를 가지는 분류 문제에 사용될 수 있는 구조
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)  # 컨볼루션 레이어 정의 (입력 채널 1, 출력 채널 20, 커널 크기 5) # 흑백 인데 5by5로...
        self.pool = nn.MaxPool2d(2, 2)  # 맥스풀링 레이어 정의 (2x2 풀링) # 4개짜리
        self.flatten = nn.Flatten()  # 텐서 평탄화
        self.fc1 = nn.Linear(2880, 50)  # 완전 연결 레이어 (입력 크기 2880, 출력 크기 50)
        self.fc2 = nn.Linear(50, 10)  # 출력 레이어 (클래스 수 10)

    def forward(self, x): # 입력 데이터 x가 모델을 통과할 때 수행되는 계산을 정의
        x = self.pool(F.relu(self.conv1(x)))  # ReLU 활성화 함수 적용 후 맥스풀링 # relu비선형 하고 pool에 넣어서 맥스풀링
        x = self.flatten(x)  # 평탄화
        x = F.relu(self.fc1(x))  # ReLU 활성화 함수 적용 # fc1 완전연결층에 집어넣기.
        x = self.fc2(x)  # 최종 출력
        return x

model = MyModel()  # 모델 인스턴스(객체) 생성

# 손실 함수 및 최적화 알고리즘 지정
# 모멘텀이 지역 최솟값에서 벗어나는데 도움이 될 수 있는 이유는, 관성의 효과로 인해 모델이 작은 지역 최솟값에서 쉽게 "빠져나와"
# 더 낮은 손실을 가지는 전역 최솟값 또는 더 큰 지역 최솟값을 찾아갈 가능성이 높아지기 때문(0은 관성이 없고 1은 크다.)
criterion = nn.CrossEntropyLoss()  # 크로스 엔트로피 손실 함수
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)  # SGD(확률적경사하강법) 최적화 알고리즘.
# lr 설명. 새 가중치=현재 가중치−(학습률(=0.01)×가중치의 기울기)
# lr은 학습률을 의미. 학습률이 너무 크면 최적점을 지나쳐 발산할 수 있고, 너무 작으면 학습이 매우 느려지거나 지역 최소값에 갇힐 수 있음
# momentum은 탈력을 받아서 실지적으로 가장 낮은지역이아닌데 빠지지 않고 더 진행할수 있도록 하는것.
# (탄력도) 지역의 최소점을 방지하는것. 더 학습을 진행 할 수 있게함

# 모델 훈련
best_val_loss = float('inf')  # 검증 손실을 추적하기 위한 변수 초기화
patience, trials = 5, 0  # 조기 종료 기준 설정
num_epochs = 20  # 에폭 수 설정
for epoch in range(num_epochs):
    model.train()  # 모델을 훈련 모드로 설정
    running_loss = 0.0
    for inputs, labels in trainloader:
        optimizer.zero_grad()  # 그래디언트 초기화
        outputs = model(inputs)  # 모델을 통한 순전파
        loss = criterion(outputs, labels)  # 손실 계산
        loss.backward()  # 역전파
        optimizer.step()  # 파라미터 업데이트
        running_loss += loss.item()

    # 검증 단계
    val_loss = 0.0
    model.eval()  # 모델을 평가 모드로 설정
    with torch.no_grad(): # 검증단계에선 no_grad()를 씀
        for inputs, labels in valloader:
            outputs = model(inputs)
            loss = criterion(outputs, labels) # 아웃풋과 실제 레이블을 비교해서 손실을 구하고
            val_loss += loss.item()

    val_loss /= len(valloader)  # 평균 검증 손실 계산
    print(f'Epoch {epoch+1}, Train Loss: {running_loss / len(trainloader)}, Val Loss: {val_loss}')

    # 검증 손실이 개선되었는지 확인하고 모델 저장
    if val_loss < best_val_loss:
        print(f'Validation Loss Decreased({best_val_loss:.6f}--->{val_loss:.6f}) \t Saving The Model')
        best_val_loss = val_loss
        trials = 0
        save_path = "/content/drive/MyDrive/hjh_kita_directory/Github/kita_231026/m6_dl/data/model/best_model.pth"
        torch.save(model.state_dict(), save_path)  # 모델 저장
    else:
        trials += 1
        if trials >= patience:  # 조기 종료 조건 충족 확인
            print("Early stopping triggered")
            break

# 최고의 모델을 불러와서 평가
model.load_state_dict(torch.load(save_path))  # 모델 상태 불러오기

# 모델 평가
correct = 0
total = 0
with torch.no_grad():  # 그래디언트 계산 비활성화
    for inputs, labels in testloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1) # predicted는 최대값의 인덱스를 의미. max_values, max_indices = torch.max(t, dim=1)
        total += labels.size(0) # 0 은 배치에 포함된 샘플 수
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total  # 정확도 계산
print(f'Accuracy on the 10000 test images: {accuracy}%')  # 정확도 출력

100%|██████████| 9912422/9912422 [00:00<00:00, 143978665.01it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 38975448.46it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 48408916.15it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 22598492.01it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch 1, Train Loss: 0.2484357401430607, Val Loss: 0.08510615367026246
Validation Loss Decreased(inf--->0.085106) 	 Saving The Model
Epoch 2, Train Loss: 0.07484213338637104, Val Loss: 0.06409525296878704
Validation Loss Decreased(0.085106--->0.064095) 	 Saving The Model
Epoch 3, Train Loss: 0.0526699892229711, Val Loss: 0.05646144534012877
Validation Loss Decreased(0.064095--->0.056461) 	 Saving The Model
Epoch 4, Train Loss: 0.042359661794888474, Val Loss: 0.0661014606631262
Epoch 5, Train Loss: 0.03267651420583328, Val Loss: 0.05865780809154843
Epoch 6, Train Loss: 0.028265214133386812, Val Loss: 0.048932730132946745
Validation Loss Decreased(0.056461--->0.048933) 	 Saving The Model
Epoch 7, Train Loss: 0.023692015175901665, Val Loss: 0.05130573351368397
Epoch 8, Train Loss: 0.01928090136599106, Val Loss: 0.048031457651535325
Validation Loss Decreased(0.048933--->0.048031) 	 Saving The Model
Epoch 9, Train Lo

torch.max(t, dim=1) 함수
- PyTorch에서 주어진 텐서 t의 각 행(row)에 대한 최대값과 해당 최대값의 인덱스를 찾는 데 사용
- dim=1은 함수가 작업을 수행할 차원을 지정

In [1]:
import torch

# 예시 텐서
t = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(t)
# 전체 텐서에서 최대값
max_val = torch.max(t)
print(max_val)  # 값: 9

# 각 행의 최대값과 해당 인덱스 찾기
max_values, max_indices = torch.max(t, dim=1)
print(max_values)  # 값: tensor([3, 6, 9])
print(max_indices)  # 인덱스: tensor([2, 2, 2])

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])
tensor(9)
tensor([3, 6, 9])
tensor([2, 2, 2])


Q. 저장된 모델 best_model.pth를 불러와서 test 데이터로 평가를 수행하세요.

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import sunmmary

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=64, shuffle=False)
testloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

class MyModel(nn.Module):
    def __init__

In [2]:
import torch
from torch.utils.data import DataLoader, random_split
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary

# MNIST 데이터셋을 위한 전처리 과정 정의
# 모델의 학습 효율성을 향상시키고, 일반적으로 신경망에서 더 나은 성능을 얻기 위해 널리 사용
transform = transforms.Compose([
    transforms.ToTensor(),  # 높이, 너비, 채널(예: RGB)의 3차원 배열을 0과 1 사이의 값으로 스케일링된 텐서로 변경
    transforms.Normalize((0.5,), (0.5,))  # 이미지를 평균 0.5, 표준편차 0.5로 정규화하여 [-1, 1] 범위로 조정
])

# MNIST 데이터셋 로드
# transform=transform: 이미지에 적용할 전처리 과정을 지정
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# 훈련 데이터셋을 훈련 및 검증 세트로 분할
train_size = int(0.8 * len(train_dataset))  # 훈련 세트 크기를 전체의 80%로 설정
val_size = len(train_dataset) - train_size  # 검증 세트 크기 계산
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])  # 분할 실행

# DataLoader는 데이터셋에서 미니 배치를 자동으로 생성하고, 이를 모델 학습이나 평가에 이용할 수 있게 해주는 유틸리티
trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True) # 데이터를 로드하기 전에 데이터셋을 무작위로 섞기
valloader = DataLoader(val_dataset, batch_size=64, shuffle=False) # 데이터의 순서가 성능 평가에 영향을 미치지 않는다.
testloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 모델 아키텍처 정의
# MyModel 클래스는 1채널 그레이스케일 이미지를 입력으로 받아, 10개의 출력 클래스를 가지는 분류 문제에 사용될 수 있는 구조
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)  # 컨볼루션 레이어 정의 (입력 채널 1, 출력 채널 20, 커널 크기 5)
        self.pool = nn.MaxPool2d(2, 2)  # 맥스풀링 레이어 정의 (2x2 풀링)
        self.flatten = nn.Flatten()  # 텐서 평탄화
        self.fc1 = nn.Linear(2880, 50)  # 완전 연결 레이어 (입력 크기 2880, 출력 크기 50)
        self.fc2 = nn.Linear(50, 10)  # 출력 레이어 (클래스 수 10)

    def forward(self, x): # 입력 데이터 x가 모델을 통과할 때 수행되는 계산을 정의
        x = self.pool(F.relu(self.conv1(x)))  # ReLU 활성화 함수 적용 후 맥스풀링
        x = self.flatten(x)  # 평탄화
        x = F.relu(self.fc1(x))  # ReLU 활성화 함수 적용
        x = self.fc2(x)  # 최종 출력
        return x

model = MyModel()  # 모델 인스턴스 생성

# 최고의 모델을 불러와서 평가
save_path = "/content/drive/MyDrive/hjh_kita_directory/Github/kita_231026/m6_dl/data/model/best_model.pth"
model.load_state_dict(torch.load(save_path))  # 모델 상태 불러오기

# 모델 평가
correct = 0
total = 0
with torch.no_grad():  # 그래디언트 계산 비활성화
    for inputs, labels in testloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1) # max_values, max_indices = torch.max(t, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total  # 정확도 계산
print(f'Accuracy on the 10000 test images: {accuracy}%')  # 정확도 출력

Accuracy on the 10000 test images: 98.91%


Q. 저장된 모델 best_model.pth를 불러와서 epochs를 추가하여 학습한 후 test 데이터로 평가를 수행하고 모델을 best_model1.pth로 저장하세요.

In [3]:
import torch
from torch.utils.data import DataLoader, random_split
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


# MNIST 데이터셋을 위한 전처리 과정 정의
# 모델의 학습 효율성을 향상시키고, 일반적으로 신경망에서 더 나은 성능을 얻기 위해 널리 사용
transform = transforms.Compose([
    transforms.ToTensor(),  # 높이, 너비, 채널(예: RGB)의 3차원 배열을 0과 1 사이의 값으로 스케일링된 텐서로 변경
    transforms.Normalize((0.5,), (0.5,))  # 이미지를 평균 0.5, 표준편차 0.5로 정규화하여 [-1, 1] 범위로 조정
])

# MNIST 데이터셋 로드
# transform=transform: 이미지에 적용할 전처리 과정을 지정
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# 훈련 데이터셋을 훈련 및 검증 세트로 분할
train_size = int(0.8 * len(train_dataset))  # 훈련 세트 크기를 전체의 80%로 설정
val_size = len(train_dataset) - train_size  # 검증 세트 크기 계산
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])  # 분할 실행

# DataLoader는 데이터셋에서 미니 배치를 자동으로 생성하고, 이를 모델 학습이나 평가에 이용할 수 있게 해주는 유틸리티
trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True) # 데이터를 로드하기 전에 데이터셋을 무작위로 섞기
valloader = DataLoader(val_dataset, batch_size=64, shuffle=False) # 데이터의 순서가 성능 평가에 영향을 미치지 않는다.
testloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 모델 아키텍처 정의
# ImprovedMyModel 클래스는 1채널 그레이스케일 이미지를 입력으로 받아, 10개의 출력 클래스를 가지는 분류 문제에 사용될 수 있는 구조
class ImprovedMyModel(nn.Module):
    def __init__(self):
        super(ImprovedMyModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)  # 컨볼루션 레이어 정의 (입력 채널 1, 출력 채널 20, 커널 크기 5)
        self.pool = nn.MaxPool2d(2, 2)  # 맥스풀링 레이어 정의 (2x2 풀링)
        self.flatten = nn.Flatten()  # 텐서 평탄화
        self.fc1 = nn.Linear(2880, 50)  # 완전 연결 레이어 (입력 크기 2880, 출력 크기 50)
        self.fc2 = nn.Linear(50, 10)  # 출력 레이어 (클래스 수 10)

    def forward(self, x): # 입력 데이터 x가 모델을 통과할 때 수행되는 계산을 정의
        x = self.pool(F.relu(self.conv1(x)))  # ReLU 활성화 함수 적용 후 맥스풀링
        x = self.flatten(x)  # 평탄화
        x = F.relu(self.fc1(x))  # ReLU 활성화 함수 적용
        x = self.fc2(x)  # 최종 출력
        return x

# 모델 아키텍처 인스턴스 생성
model = ImprovedMyModel()
# 모델 객체 이름이 바뀔순 있지만, 사용자 함수의 가중치 구조를 바꿀순 없음.(물론 고급에서는 바꿈) 바꾸면 오류남. 대신 에폭을 늘리면됨

# 저장된 state_dict 불러오기
state_dict = torch.load('/content/drive/MyDrive/hjh_kita_directory/Github/kita_231026/m6_dl/data/model/best_model.pth')

# 불러온 state_dict를 모델에 로드
model.load_state_dict(state_dict)

# 모델 구조 출력
print(model)

# 손실 함수 및 최적화 알고리즘 지정
criterion = nn.CrossEntropyLoss()  # 크로스 엔트로피 손실 함수
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)  # SGD 최적화 알고리즘

# epochs 추가 학습을 위한 설정
num_additional_epochs = 10
for epoch in range(num_additional_epochs):
    model.train()  # 모델을 훈련 모드로 설정
    running_loss = 0.0
    for inputs, labels in trainloader:
        optimizer.zero_grad()  # 그래디언트 초기화
        outputs = model(inputs)  # 모델을 통한 순전파
        loss = criterion(outputs, labels)  # 손실 계산
        loss.backward()  # 역전파
        optimizer.step()  # 파라미터 업데이트
        running_loss += loss.item()
    # 여기서는 간단화를 위해 검증 단계는 생략했습니다. 필요에 따라 추가하세요.
    print(f'Epoch {epoch+1}, Train Loss: {running_loss / len(trainloader)}')


# 모델 평가
correct = 0
total = 0
with torch.no_grad():  # 그래디언트 계산 비활성화
    for inputs, labels in testloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total  # 정확도 계산
print(f'Accuracy on the 10000 test images: {accuracy}%')  # 정확도 출력

import os
# Define the save path
save_path = '/content/drive/MyDrive/hjh_kita_directory/Github/kita_231026/m6_dl/data/model/best_model1.pth'

# Ensure directory exists
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# 모델의 매개변수(가중치와 편향)만 저장
torch.save(model.state_dict(), save_path)
print("Saved PyTorch Model State to model.pth")

ImprovedMyModel(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=2880, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)
Epoch 1, Train Loss: 0.021417105742545878
Epoch 2, Train Loss: 0.013603444273083975
Epoch 3, Train Loss: 0.009824811068200991
Epoch 4, Train Loss: 0.007183098133443612
Epoch 5, Train Loss: 0.005815988393776934
Epoch 6, Train Loss: 0.0037090883138686573
Epoch 7, Train Loss: 0.0021656318516276468
Epoch 8, Train Loss: 0.0016981516563312955
Epoch 9, Train Loss: 0.0013744766928127016
Epoch 10, Train Loss: 0.0011055277176904687
Accuracy on the 10000 test images: 98.85%
Saved PyTorch Model State to model.pth
